# Example of reading / saving data from / to GEFF

This example illustrates how we can save the tracked data into a GEFF (Graph Exchange File Format).

In [27]:
from laptrack import LapTrack
from laptrack import data_conversion
from laptrack import datasets
import geff
import tempfile
from pathlib import Path
import os
import shutil

In [2]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    splitting_cutoff=15**2,
    merging_cutoff=15**2,
)

In [3]:
spots_df = datasets.simple_tracks()
coordinate_cols = ["position_x", "position_y"]
frame_col = "frame"
spots_df.head()

,frame,position_x,position_y
0,0,178.412575,185.188661
1,1,236.963642,219.971473
2,1,185.175899,185.188661
3,2,190.006845,182.483331
4,2,190.200083,188.280466


In [4]:
track_df, split_df, merge_df = lt.predict_dataframe(
    spots_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
)

track_df.head()

,frame,position_x,position_y,tree_id,track_id
0,0,178.412575,185.188661,0,0
1,1,236.963642,219.971473,1,1
2,1,185.175899,185.188661,0,0
3,2,190.006845,182.483331,0,2
4,2,190.200083,188.280466,0,3


In [17]:
geff_networks = data_conversion.dataframes_to_geff_networkx(
    track_df,
    split_df,
    merge_df,
    coordinate_cols=coordinate_cols + ["track_id"],
    frame_col=frame_col,
)
for track_id in geff_networks.lineage_tree.nodes:
    geff_networks.lineage_tree.nodes[track_id]["length"] = len(
        list(
            filter(
                lambda n: n[1]["track_id"] == track_id,
                geff_networks.tree.nodes(data=True),
            )
        )
    )

In [18]:
print(geff_networks.tree.nodes[0])
print(geff_networks.lineage_tree.nodes[0])

{'frame': 0, 'position_x': np.float64(178.4125746427294), 'position_y': np.float64(185.18866073671265), 'track_id': np.float64(0.0)}
{'length': 2}


In [30]:
temp_dir = tempfile.gettempdir()
f = Path(temp_dir) / "tree.geff"
shutil.rmtree(f)
geff.write_nx(
    geff_networks.tree,
    f,
    axis_names=[frame_col] + coordinate_cols,
    axis_types=["time", "space", "space"],
    axis_units=["frame", "px", "px"],
)
geff.validate(f)

geff.write_nx(
    geff_networks.lineage_tree,
    f / "linage_tree.geff",
)
!ls {f / "linage_tree.geff" / "nodes" / "props"}
!ls {temp_dir}/"tree.geff"

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter'

length
edges  linage_tree.geff  nodes
